In [2]:
#importing packages

In [55]:
import pandas as pd
from os import mkdir, path
from datetime import timedelta
import time
import twint
import nest_asyncio
nest_asyncio.apply()
import csv
import re
from collections import defaultdict

In [ ]:
#defining search function

In [ ]:
def twint_search(since, until, csv_name, coordinate):
    # scrape tweets since(day) to until(day) within the geo(coordinates) with the english Language tag, store tweets in 
    # a csv file with the given csv_name
    c = twint.Config()
    c.Since = since
    c.Until = until
    c.Geo = coordinate
    c.Count = True
    c.Lang = "en"
    c.Hide_output = True
    c.Store_csv = True
    c.Output = csv_name
    c.Debug = True
    
    try:
        twint.run.Search(c)
    except(KeyboardInterrupt, SystemExit):
        raise
    except:
        print("Problem with %s"%since)

In [1]:
# defining scraping function that scrapes within a circle defined by each element in a list of coordinates for the days 
# between since & until and saves tweets of each day in a separate csv file, and saves the files in a folder titled 
# state_name

In [ ]:
def my_scraper(since, until, state_name, coordinates):
    #create a folder with the name state_name
    dirname = state_name
    try:
        mkdir(dirname)
        print("Directory", dirname, "created")
    except FileExistsError:
        print("Directory", dirname, "exists")
    
    #convert string time range into pandas datetime
    daterange = pd.date_range(since, until)
    for start_date in daterange:
        since = start_date.strftime("%Y-%m-%d")
        until = (start_date + timedelta(1)).strftime("%Y-%m-%d")
        
        #create a csv file with the since_date as name
        print("Getting data for %s"%since)
        csv_name = "%s.csv"%since
        csv_name = path.join(dirname, csv_name)
        
        #loop through the list of given coordinates and scrape tweets since_day to until_day
        for coordinate in coordinates:
            twint_search(since, until, csv_name, coordinate)
            
        time.sleep(60)

In [3]:
#loading coordinates

In [21]:
cf = pd.read_csv("map circles.csv")

In [52]:
cf.set_index("Zone", inplace = True)

In [87]:
coordinates = defaultdict(list)
for i in cf.index:
    coordinates[i.split("_")[0]].append("{},{}km".format((cf.at[i, "Coordinates"]), \
                                                          round(float((cf.at[i, "Radius"]).split(" ")[0])/1000)))

In [88]:
coordinates

defaultdict(list,
            {'Himachal': ['31.873260,77.218891,100km',
              '30.744087,77.442958,25km'],
             'Punjab': ['31.516029,75.334976,63km',
              '30.470609,74.816149,62km',
              '30.726231,75.991178,45km',
              '30.072987,75.687661,32km',
              '32.223745,75.630766,20km',
              '30.309005,76.434785,17km'],
             'Haryana': ['29.574939,74.938006,36km',
              '29.378815,75.712523,43km',
              '29.118284,76.906444,22km',
              '28.696073,76.311358,52km',
              '29.714145,76.602625,50km',
              '30.346627,77.052975,33km',
              '29.894954,77.398828,25km'],
             'Delhi': ['28.519553,77.250906,41km'],
             'UP': ['29.250640,77.668503,49km',
              '28.765447,78.580363,54km',
              '27.642454,78.206687,74km',
              '28.011467,79.778266,83km',
              '25.505168,78.886895,43km',
              '26.344840,80.250508,106km',
    

In [89]:
#iteratively scraping through a dictionary with state_names as keys, and a list of coordinates for various circles 
#within the state as values

In [ ]:
for key, value in coordinates.items():
    my_scraper("2020-01-22", "2020-05-30", key, value)